In [ ]:
# Tests for Horizon attributes
import sys
import warnings
import numpy as np

warnings.filterwarnings('ignore')

sys.path.append('../../') # for running py-script
sys.path.append('../../..') # for running this notebook directly

from seismiqb import Field, SeismicGeometry, Horizon

In [ ]:
# CUBE_PATH = 'path/to/cube'
# HORIZON_PATH = 'path/to/horizon'

# FIGSIZE = (12, 7)

ATTRS_LIST = ['amplitudes', 'depths', 'instant_phases',
              'instant_amplitudes', 'fourier', 'wavelet', 'masks']

In [ ]:
%%time
geometry = SeismicGeometry(CUBE_PATH)
field = Field(geometry)

# init from_file
horizon = Horizon(HORIZON_PATH, field=field)

# make it pretty:
horizon.filter()

# Operations with FILL_VALUE

In [ ]:
%%time
horizon_copy = horizon.__copy__()

# Change fill_value to a new one
new_fill_value = -10

# New and old fill_values amount before filling
n_fill_value = np.sum(horizon_copy.matrix == new_fill_value)
n_absent = np.sum(horizon_copy.matrix == horizon_copy.FILL_VALUE)

horizon_copy.matrix_fill_to_num(matrix=horizon_copy.matrix, value=new_fill_value)

# New fill_values amount after filling
new_n_fill_value = np.sum(horizon_copy.matrix == new_fill_value)

assert new_n_fill_value == n_fill_value + n_absent, "matrix_fill_to_num test failed"

In [ ]:
%%time
# Reverse filling: new fill_value to old
horizon_copy.matrix_num_to_fill(matrix=horizon_copy.matrix, value=new_fill_value)

old_n_fill_value = np.sum(horizon_copy.matrix == horizon_copy.FILL_VALUE)

assert old_n_fill_value == new_n_fill_value, "matrix_num_to_fill test failed"

# Matrix operations

In [ ]:
%%time
# Normalize
modes = ['min-max', 'mean-std'] # False - docstring contains this option but it is extra

for mode in modes:
    horizon_copy = horizon.__copy__()
    full_matrix = horizon_copy.matrix_put_on_full(horizon_copy.matrix)

    horizon_copy.matrix_normalize(matrix=full_matrix, mode=mode)
    
    print(f"{mode} mode was successfully checked.")

# Properties

In [ ]:
%%time
_ = horizon.binary_matrix
_ = horizon.presence_matrix
_ = horizon.borders_matrix
_ = horizon.boundaries_matrix
_ = horizon.filled_matrix

# Get attributes

In [ ]:
%%time
sides = [-1, 1]

for side in sides:
    _ = horizon.get_zerocrossings(side=side)
    
    print(f"{side} side was successfully checked.")

In [ ]:
%%time
_ = horizon.get_median_diff_map()
_ = horizon.get_gradient_map()
_ = horizon.get_spikes_map()

# Loading attributes

In [ ]:
%%time
metrics = horizon.load_attribute('metrics', metric='local_corrs', normalize='min-max')

In [ ]:
%%time
N_COLS = 2
N_ROWS = int(np.ceil(len(ATTRS_LIST)/N_COLS))
IMAGE_HEIGHT = 7

horizon.show(ATTRS_LIST, separate=True, nrows=N_ROWS, ncols=N_COLS,
             figsize=(21, N_ROWS*IMAGE_HEIGHT))